##**25DLS037_AKSHIT_SHARMA**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Load data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Build model
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(x_train, y_train, epochs=10, validation_split=0.2)

# Plot accuracy and loss
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='val')
plt.title('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.title('Loss')
plt.legend()
plt.show()

# Evaluate
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.utils import to_categorical
import numpy as np

# Preprocess
x_train, x_test = x_train.astype("float32"), x_test.astype("float32")
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

# Load base model
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(32, 32, 3))
base_model.trainable = False

# Add head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train
history = model.fit(x_train, y_train_cat, validation_split=0.2, epochs=5)

# Evaluate
loss, acc = model.evaluate(x_test, y_test_cat)
print(f"Test Accuracy: {acc:.4f}")

# Show 5 misclassified images
preds = model.predict(x_test)
misclassified = np.where(np.argmax(preds, axis=1) != y_test.reshape(-1))[0][:5]

plt.figure(figsize=(10, 5))
for i, idx in enumerate(misclassified):
    plt.subplot(1, 5, i+1)
    plt.imshow(x_test[idx])
    plt.title(f"Pred: {np.argmax(preds[idx])}\nTrue: {y_test[idx][0]}")
    plt.axis('off')
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Sample text
text = "hello world! this is a simple character-level RNN example."
chars = sorted(set(text))
char2idx = {c: i for i, c in enumerate(chars)}
idx2char = np.array(chars)
text_as_int = np.array([char2idx[c] for c in text])

# Create sequences
seq_length = 10
examples_per_epoch = len(text) - seq_length
X = []
y = []
for i in range(examples_per_epoch):
    X.append(text_as_int[i:i+seq_length])
    y.append(text_as_int[i+seq_length])
X = np.array(X)
y = np.array(y)

# Build LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(chars), 16, input_length=seq_length),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(len(chars), activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

# Train
history = model.fit(X, y, epochs=20)

# Generate text
def generate_text(seed, length=50):
    for _ in range(length):
        input_seq = np.array([char2idx[c] for c in seed[-seq_length:]]).reshape(1, -1)
        pred = model.predict(input_seq, verbose=0)
        next_char = idx2char[np.argmax(pred)]
        seed += next_char
    return seed

print(generate_text("hello worl"))

# Plot loss
plt.plot(history.history['loss'])
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()


In [ ]:
from transformers import pipeline

# Load summarizer
summarizer = pipeline("summarization", model="t5-small", tokenizer="t5-small")

text = """Artificial intelligence is the simulation of human intelligence processes by machines,
especially computer systems. These processes include learning, reasoning, and self-correction.
AI is being applied across industries, from healthcare to finance, to create smarter applications."""

# Default prompt
summary1 = summarizer("summarize: " + text, max_length=50, min_length=20, do_sample=False)[0]['summary_text']

# Modified prompt
summary2 = summarizer("summarize news: " + text, max_length=50, min_length=20, do_sample=False)[0]['summary_text']
summary3 = summarizer("summarize this article: " + text, max_length=50, min_length=20, do_sample=False)[0]['summary_text']

print("Summary 1:\n", summary1)
print("Summary 2:\n", summary2)
print("Summary 3:\n", summary3)


In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
import matplotlib.pyplot as plt

# Load data
(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

# Encoder
input_img = Input(shape=(28, 28, 1))
x = Conv2D(16, (3,3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2,2), padding='same')(x)
x = Conv2D(8, (3,3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2,2), padding='same')(x)

# Decoder
x = Conv2D(8, (3,3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2,2))(x)
x = Conv2D(16, (3,3), activation='relu')(x)
x = UpSampling2D((2,2))(x)
decoded = Conv2D(1, (3,3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.fit(x_train, x_train, epochs=10, validation_data=(x_test, x_test))

# Plot results
decoded_imgs = autoencoder.predict(x_test)
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # original
    ax = plt.subplot(2, n, i+1)
    plt.imshow(x_test[i].reshape(28,28), cmap='gray')
    plt.axis('off')

    # reconstructed
    ax = plt.subplot(2, n, i+1+n)
    plt.imshow(decoded_imgs[i].reshape(28,28), cmap='gray')
    plt.axis('off')
plt.show()

# Bottleneck shape
encoder = Model(input_img, encoded)
bottleneck = encoder.predict(x_test)
print("Bottleneck shape:", bottleneck.shape)
